In [ ]:
from pathlib import Path

import numpy as np
import xarray as xr
from siphon.catalog import TDSCatalog

from atm_forcing import CF_ROMS, generate_catalog_urls, get_ds, get_ds_roms, reshape_to_full_year  # noqa: F401


### Download, preprocess, save 

In [ ]:
LAT_NEW = np.arange(58.9, 60, 0.02)
LON_NEW = np.arange(10.1, 11.1, 0.02)

In [ ]:
file_path_grid = Path.home() / "dump_fram_nn9297k" / "ROHO800_grid_fix5.nc"
ds_grid = xr.open_dataset(file_path_grid)

In [ ]:
regridder = None
dss = []
timestamps = []
parameters = [x[0] for x in CF_ROMS]
for date_and_time, catalog_url in generate_catalog_urls(start_year=2005, end_year=2021):
    timestamp = date_and_time.strftime("%Y%m%d")
    file_path = Path.home() / "NORA3" / f"{timestamp}.nc"
    if file_path.exists():
        print(f"File {file_path} exists for {date_and_time}.")
        continue
    print(f"Processing: {date_and_time}.")
    cat = TDSCatalog(catalog_url)
    urls = [v.access_urls["opendap"] for k, v in cat.datasets.items() if "_fp" in k]
    ds = xr.open_mfdataset(urls, combine="by_coords", compat="no_conflicts", data_vars="all")
    ds = ds[parameters]
    # regridder, ds = get_ds(regridder, ds, LAT_NEW, LON_NEW)
    regridder, ds = get_ds_roms(regridder, ds, ds_grid.lat_rho, ds_grid.lon_rho)
    dss.append(ds)
    timestamps.append(timestamp)
    if len(dss) > 3:  # there should be 4 files per day.
        assert len(set(timestamps)) <= 1
        ds = xr.combine_by_coords(dss, coords=["time"], join="outer", combine_attrs="override", compat="no_conflicts")
        ds = ds.sel(time=~ds.get_index("time").duplicated())
        print(f"Saving to {file_path}.")
        ds.to_netcdf(file_path, encoding={var: {"zlib": True, "complevel": 5} for var in ds.data_vars})
        print("Saving done.")
        dss = []
        timestamps = []

### Checkup

In [ ]:
pattern = str(Path.home() / "FjordSim_data" / "NORA3" / "[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9].nc")
ds = xr.open_mfdataset(pattern, combine="by_coords")

In [ ]:
ds = reshape_to_full_year(ds)

In [ ]:
bad_steps = ds.time.diff("time") != np.timedelta64(1, "h")
print("Bad steps at positions:", bad_steps.where(bad_steps, drop=True))

In [ ]:
ds

In [ ]:
ds.to_netcdf(
    Path.home() / "FjordSim_data" / "NORA3" / "NORA3.nc",
    encoding={var: {"zlib": True, "complevel": 5} for var in ds.data_vars},
)

### Saving atm parameters separately

In [ ]:
for var_name, da in ds.data_vars.items():
    da = da.rename(var_name)
    filename = f"{var_name}.nc"
    da.to_netcdf(
        Path.home() / "FjordSim_data" / "NORA3" / filename, encoding={var_name: {"zlib": True, "complevel": 5}}
    )
    print(f"{filename} saved.")